In [1]:
!pip install praw vaderSentiment

In [83]:
import requests
import json
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

In [4]:
api_key = "VIK1I1U7QM7V0K5J"

url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey={api_key}"
r = requests.get(url)
data = r.json()

#data

{'items': '50',
 'sentiment_score_definition': 'x <= -0.35: Bearish; -0.35 < x <= -0.15: Somewhat-Bearish; -0.15 < x < 0.15: Neutral; 0.15 <= x < 0.35: Somewhat_Bullish; x >= 0.35: Bullish',
 'relevance_score_definition': '0 < x <= 1, with a higher score indicating higher relevance.',
 'feed': [{'title': 'Trump\'s "Reciprocal Tariffs": Why Major AI Stocks Apple, Meta, and Alphabet  ( Google )  Are Feeling the Heat Today',
   'url': 'https://www.fool.com/investing/2025/04/03/trumps-reciprocal-tariffs-why-major-ai-stocks-appl/',
   'time_published': '20250403T182712',
   'authors': ['Bram Berkowitz'],
   'summary': 'The whole U.S. stock market is feeling the pain from the sweeping new tariffs that President Trump announced Wednesday after the market closed. As of 1:10 p.m. ET Thursday, the blue chip Dow Jones Industrial Average had fallen by over 1,300 points ( 3.1% ) , the broad market S&P 500 was off by ...',
   'banner_image': 'https://g.foolcdn.com/editorial/images/813576/trump-offic

In [ ]:
%run ETL.ipynb

In [95]:
# Stock ticker symbol you want sentiment data for (e.g., AAPL for Apple)
ticker = "AAPL"
limit = 1000
beg_time = "20200410T0130"
end_time = "20250401T0130"

# Construct the URL
base_url = "https://www.alphavantage.co/query"
params = {
    "function": "NEWS_SENTIMENT",
    "tickers": ticker,
    "apikey": api_key,
    "limit": limit,
    "time_from": beg_time,
    "time_to": end_time
}

# Make the request
response = requests.get(base_url, params=params)
data = response.json()

# Print or process the results
#print(json.dumps(data, indent=4))

In [39]:
articles = []
    
for item in data.get('feed', []):
        # Extract the sentiment data for AAPL specifically
    ticker_sentiments = {sentiment['ticker']: sentiment for sentiment in item.get('ticker_sentiment', [])}
    apple_sentiment = ticker_sentiments.get('AAPL', {})
        
    articles.append({
        'time_published': item.get('time_published'),
        'title': item.get('title'),
        'overall_sentiment_score': item.get('overall_sentiment_score'),
        'overall_sentiment_label': item.get('overall_sentiment_label'),
        'apple_relevance_score': apple_sentiment.get('relevance_score'),
        'apple_sentiment_score': apple_sentiment.get('ticker_sentiment_score'),
        'apple_sentiment_label': apple_sentiment.get('ticker_sentiment_label')
    })

df = pd.DataFrame(articles)
if not df.empty and 'time_published' in df.columns:
    df['time_published'] = pd.to_datetime(df['time_published'])
    df.set_index('time_published', inplace=True)

sentiment_df = df

sentiment_df

,title,overall_sentiment_score,overall_sentiment_label,apple_relevance_score,apple_sentiment_score,apple_sentiment_label
time_published,,,,,,
2025-03-31 23:15:10,Here's How Much $100 Invested In Apple 5 Years...,0.250000,Somewhat-Bullish,0.941873,0.382786,Bullish
2025-03-31 21:47:00,Navigating Investment in Uncertain Markets,0.225265,Somewhat-Bullish,0.16849,0.241871,Somewhat-Bullish
2025-03-31 17:49:00,How Cheap Are Big Tech Stocks?,0.253909,Somewhat-Bullish,0.146025,0.013941,Neutral
2025-03-31 15:00:51,Investigating Apple's Standing In Technology H...,0.275692,Somewhat-Bullish,0.469099,0.360504,Bullish
2025-03-31 14:37:00,"The Market's Volatility, CoreWeave's IPO, Bitc...",0.065454,Neutral,0.012479,0.076503,Neutral
...,...,...,...,...,...,...
2025-02-06 03:56:00,Music Production Software Market to Grow by US...,0.360863,Bullish,0.023957,0.04085,Neutral
2025-02-06 03:45:00,Television Market to Grow by USD 73.1 Billion ...,0.207262,Somewhat-Bullish,0.022431,0.0,Neutral
2025-02-06 03:25:27,Amazon's 'Remarkable Alexa' Is Coming-Here's W...,0.177666,Somewhat-Bullish,0.290033,0.347552,Somewhat-Bullish


In [40]:
sentiment_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 673 entries, 2025-03-31 23:15:10 to 2025-02-06 01:19:19
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   title                    673 non-null    object 
 1   overall_sentiment_score  673 non-null    float64
 2   overall_sentiment_label  673 non-null    object 
 3   apple_relevance_score    673 non-null    object 
 4   apple_sentiment_score    673 non-null    object 
 5   apple_sentiment_label    673 non-null    object 
dtypes: float64(1), object(5)
memory usage: 36.8+ KB


In [41]:
sentiment_df.dtypes

title                       object
overall_sentiment_score    float64
overall_sentiment_label     object
apple_relevance_score       object
apple_sentiment_score       object
apple_sentiment_label       object
dtype: object

In [42]:
numeric_columns = ['overall_sentiment_score', 'apple_relevance_score', 'apple_sentiment_score']
for col in numeric_columns:
    # Convert to string, strip any whitespace, then convert to float
    sentiment_df[col] = pd.to_numeric(sentiment_df[col], errors='coerce')
    
    # Check for any remaining NaN values after conversion
    if sentiment_df[col].isna().any():
        print(f"Warning: {sentiment_df[col].isna().sum()} NaN values in {col}")

In [43]:
sentiment_df.dtypes

title                       object
overall_sentiment_score    float64
overall_sentiment_label     object
apple_relevance_score      float64
apple_sentiment_score      float64
apple_sentiment_label       object
dtype: object

In [44]:
daily_sentiment = sentiment_df.resample('D').agg({
    'overall_sentiment_score': 'mean',
    'apple_sentiment_score': 'mean',
    'apple_relevance_score': 'mean',
    # Count of articles per day might be useful
    'title': 'count'
}).rename(columns={'title': 'article_count'})

In [48]:
daily_sentiment.head()

,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count
time_published,,,,
2025-02-06,0.235998,0.129386,0.183612,22
2025-02-07,0.251546,0.180544,0.353607,17
2025-02-08,0.282259,0.270557,0.421006,7
2025-02-09,0.171041,0.092061,0.309307,6
2025-02-10,0.246183,0.183048,0.232151,14


In [49]:
daily_sentiment.tail()

,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count
time_published,,,,
2025-03-27,0.175140,0.215188,0.525012,12
2025-03-28,0.190572,0.097086,0.209999,14
2025-03-29,0.138288,0.128809,0.211941,4
2025-03-30,0.196711,0.229540,0.306971,4
2025-03-31,0.194707,0.158460,0.244949,13


In [46]:
daily_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 54 entries, 2025-02-06 to 2025-03-31
Freq: D
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall_sentiment_score  54 non-null     float64
 1   apple_sentiment_score    54 non-null     float64
 2   apple_relevance_score    54 non-null     float64
 3   article_count            54 non-null     int64  
dtypes: float64(3), int64(1)
memory usage: 2.1 KB


In [64]:
stock_price_df = df
stock_price_df = stock_price_df.reset_index()
stock_price_df.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2025-04-03,205.540,207.49,201.25,203.19,102436930.0
1,2025-04-02,221.315,225.19,221.02,223.89,35905904.0
2,2025-04-01,219.805,223.68,218.90,223.19,36412740.0
3,2025-03-31,217.005,225.62,216.23,222.13,65299321.0
4,2025-03-28,221.670,223.81,217.68,217.90,39818617.0


In [65]:
daily_sentiment = daily_sentiment.reset_index()
daily_sentiment = daily_sentiment.rename(columns={'time_published': 'date'})
daily_sentiment.head()

,date,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count
0,2025-02-06,0.235998,0.129386,0.183612,22
1,2025-02-07,0.251546,0.180544,0.353607,17
2,2025-02-08,0.282259,0.270557,0.421006,7
3,2025-02-09,0.171041,0.092061,0.309307,6
4,2025-02-10,0.246183,0.183048,0.232151,14


In [66]:
daily_sentiment = daily_sentiment.sort_values(by='date', ascending=True)
stock_price_df = stock_price_df.sort_values(by='date', ascending=True)

print(stock_price_df.head())
print(daily_sentiment.head())

           date  1. open  2. high  3. low  4. close  5. volume
6394 1999-11-01    80.00    80.69   77.37     77.62  2487300.0
6393 1999-11-02    78.00    81.69   77.31     80.25  3564600.0
6392 1999-11-03    81.62    83.25   81.00     81.50  2932700.0
6391 1999-11-04    82.06    85.37   80.62     83.62  3384700.0
6390 1999-11-05    84.62    88.37   84.00     88.31  3721500.0
        date  overall_sentiment_score  apple_sentiment_score  \
0 2025-02-06                 0.235998               0.129386   
1 2025-02-07                 0.251546               0.180544   
2 2025-02-08                 0.282259               0.270557   
3 2025-02-09                 0.171041               0.092061   
4 2025-02-10                 0.246183               0.183048   

   apple_relevance_score  article_count  
0               0.183612             22  
1               0.353607             17  
2               0.421006              7  
3               0.309307              6  
4               0.232151   

In [67]:
stock_price_df.columns

Index(['date', '1. open', '2. high', '3. low', '4. close', '5. volume'], dtype='object')

In [68]:
stock_price_df['date'] = pd.to_datetime(stock_price_df['date'])
daily_sentiment['date'] = pd.to_datetime(daily_sentiment['date'])

min_date = daily_sentiment['date'].min()
max_date = daily_sentiment['date'].max()

stock_price_df = stock_price_df[(stock_price_df['date'] >= min_date) & (daily_sentiment['date'] <= max_date)]

print(stock_price_df.head())
print(daily_sentiment.head())

         date  1. open  2. high   3. low  4. close   5. volume
39 2025-02-06  231.285  233.800  230.425    233.22  29925349.0
38 2025-02-07  232.600  234.000  227.260    227.63  39707224.0
37 2025-02-10  229.570  230.585  227.200    227.65  33115645.0
36 2025-02-11  228.200  235.230  228.130    232.62  53718362.0
35 2025-02-12  231.200  236.960  230.680    236.87  45243292.0
        date  overall_sentiment_score  apple_sentiment_score  \
0 2025-02-06                 0.235998               0.129386   
1 2025-02-07                 0.251546               0.180544   
2 2025-02-08                 0.282259               0.270557   
3 2025-02-09                 0.171041               0.092061   
4 2025-02-10                 0.246183               0.183048   

   apple_relevance_score  article_count  
0               0.183612             22  
1               0.353607             17  
2               0.421006              7  
3               0.309307              6  
4               0.232151   

/var/folders/8w/6jy8grcd51v05fp3wl260f7w0000gn/T/ipykernel_39470/3142639716.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  stock_price_df = stock_price_df[(stock_price_df['date'] >= min_date) & (daily_sentiment['date'] <= max_date)]


In [71]:
merged_df = pd.merge(stock_price_df, daily_sentiment, on='date', how='inner')

merged_df.head(15)

,date,1. open,2. high,3. low,4. close,5. volume,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count
0,2025-02-06,231.285,233.8000,230.4250,233.22,29925349.0,0.235998,0.129386,0.183612,22
1,2025-02-07,232.600,234.0000,227.2600,227.63,39707224.0,0.251546,0.180544,0.353607,17
2,2025-02-10,229.570,230.5850,227.2000,227.65,33115645.0,0.246183,0.183048,0.232151,14
3,2025-02-11,228.200,235.2300,228.1300,232.62,53718362.0,0.172520,0.173904,0.479615,10
4,2025-02-12,231.200,236.9600,230.6800,236.87,45243292.0,0.187009,0.160707,0.390377,22
5,2025-02-13,236.910,242.3399,235.5700,241.53,53614054.0,0.186973,0.152460,0.351148,13
6,2025-02-14,241.250,245.5500,240.9900,244.60,40896227.0,0.192267,0.130857,0.366558,20
7,2025-02-18,244.150,245.1800,241.8400,244.47,48822491.0,0.195193,0.142494,0.329926,18
8,2025-02-19,244.660,246.0100,243.1604,244.87,32204215.0,0.178265,0.118498,0.222341,20
9,2025-02-20,244.940,246.7800,244.2900,245.83,32316907.0,0.187071,0.159299,0.310891,21


In [72]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date                     37 non-null     datetime64[ns]
 1   1. open                  37 non-null     float64       
 2   2. high                  37 non-null     float64       
 3   3. low                   37 non-null     float64       
 4   4. close                 37 non-null     float64       
 5   5. volume                37 non-null     float64       
 6   overall_sentiment_score  37 non-null     float64       
 7   apple_sentiment_score    37 non-null     float64       
 8   apple_relevance_score    37 non-null     float64       
 9   article_count            37 non-null     int64         
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 3.0 KB


In [79]:
#feature engineering

merged_df['positive_sentiment'] = merged_df['overall_sentiment_score'].apply(lambda x: 1 if x > 0 else 0)
merged_df['negative_sentiment'] = merged_df['overall_sentiment_score'].apply(lambda x: 1 if x < 0 else 0)
merged_df['neutral_sentiment'] = merged_df['overall_sentiment_score'].apply(lambda x: 1 if x == 0 else 0)
merged_df['lag_1'] = merged_df['4. close'].shift(1)
merged_df['lag_2'] = merged_df['4. close'].shift(2)
merged_df['lag_sentiment'] = merged_df['overall_sentiment_score'].shift(1)
merged_df['7_day_ma'] = merged_df['4. close'].rolling(window=7).mean()
merged_df['price_change'] = merged_df['4. close'] - merged_df['lag_1']
merged_df['pct_change'] = merged_df['price_change'] / merged_df['lag_1']
merged_df['sentiment_3day_avg'] = merged_df['overall_sentiment_score'].rolling(window=3).mean()
merged_df['day_of_week'] = pd.to_datetime(merged_df['date']).dt.dayofweek
merged_df.dropna(inplace=True)
merged_df.head()

,date,1. open,2. high,3. low,4. close,5. volume,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count,...,neutral_sentiment,lag_close,7_day_ma,lag_1,lag_2,lag_sentiment,price_change,pct_change,sentiment_3day_avg,day_of_week
24,2025-03-13,215.95,216.8394,208.42,209.68,61368330.0,0.198543,0.059364,0.167210,12,...,0,216.98,226.445714,216.98,220.84,0.183121,-7.30,-0.033644,0.166815,3
25,2025-03-14,211.25,213.9500,209.58,213.49,60107582.0,0.243122,0.258127,0.309141,7,...,0,209.68,223.267143,209.68,216.98,0.198543,3.81,0.018171,0.208262,4
26,2025-03-17,213.31,215.2200,209.97,214.00,48073426.0,0.153616,0.131720,0.475376,9,...,0,213.49,220.220000,213.49,209.68,0.243122,0.51,0.002389,0.198427,0
27,2025-03-18,214.16,215.1500,211.49,212.69,42432426.0,0.196049,0.143111,0.293222,11,...,0,214.00,216.451429,214.00,213.49,0.153616,-1.31,-0.006121,0.197596,1
28,2025-03-19,214.22,218.7600,213.75,215.24,54385391.0,0.161412,0.035105,0.199260,14,...,0,212.69,214.702857,212.69,214.00,0.196049,2.55,0.011989,0.170359,2


In [82]:
merged_df['target'] = merged_df['4. close'].shift(-1)  # Predict tomorrow's price
merged_df.dropna(inplace=True)  # Remove rows with NaNs caused by shifting or rolling
merged_df.head()

,date,1. open,2. high,3. low,4. close,5. volume,overall_sentiment_score,apple_sentiment_score,apple_relevance_score,article_count,...,lag_close,7_day_ma,lag_1,lag_2,lag_sentiment,price_change,pct_change,sentiment_3day_avg,day_of_week,target
24,2025-03-13,215.95,216.8394,208.42,209.68,61368330.0,0.198543,0.059364,0.167210,12,...,216.98,226.445714,216.98,220.84,0.183121,-7.30,-0.033644,0.166815,3,213.49
25,2025-03-14,211.25,213.9500,209.58,213.49,60107582.0,0.243122,0.258127,0.309141,7,...,209.68,223.267143,209.68,216.98,0.198543,3.81,0.018171,0.208262,4,214.00
26,2025-03-17,213.31,215.2200,209.97,214.00,48073426.0,0.153616,0.131720,0.475376,9,...,213.49,220.220000,213.49,209.68,0.243122,0.51,0.002389,0.198427,0,212.69
27,2025-03-18,214.16,215.1500,211.49,212.69,42432426.0,0.196049,0.143111,0.293222,11,...,214.00,216.451429,214.00,213.49,0.153616,-1.31,-0.006121,0.197596,1,215.24
28,2025-03-19,214.22,218.7600,213.75,215.24,54385391.0,0.161412,0.035105,0.199260,14,...,212.69,214.702857,212.69,214.00,0.196049,2.55,0.011989,0.170359,2,214.10


In [84]:
merged_df['target'] = (merged_df['4. close'].shift(-1) > merged_df['4. close']).astype(int)
merged_df.dropna(inplace=True)  # Clean any NaNs from shift
merged_df['target'].head()

24    1
25    1
26    0
27    1
28    0
Name: target, dtype: int64

In [90]:
merged_df.sort_values("date", inplace=True)
print(merged_df.shape)

(11, 23)


In [91]:
model  = RandomForestClassifier(n_estimators=200, min_samples_split=100, random_state=1)

train = merged_df.iloc[:-3]
test = merged_df.iloc[-3:]


predictors = [
    '4. close', 'overall_sentiment_score', '7_day_ma', 
    'positive_sentiment', 'negative_sentiment',
    'lag_1', 'lag_2', 'price_change', 'pct_change',
    'sentiment_3day_avg', 'day_of_week'
]

model.fit(train[predictors], train["target"])



RandomForestClassifier(min_samples_split=100, n_estimators=200, random_state=1)

In [92]:
predictions = model.predict(test[predictors])

predictions

array([1, 1, 1])

In [93]:
predictions = pd.Series(predictions, index=test.index)
predictions

32    1
33    1
34    1
dtype: int64

In [94]:
precision_score(test["target"], predictions)

0.3333333333333333